# 1. 사전 세팅

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%cd "/content/drive/MyDrive/데이터 분석/projects/ML_portfolio/10_kleague_final_pass_prediction"

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 2. 문제 정의

---

    현대 축구에서 승패는 개별 선수의 기량을 넘어, 유기적인 팀 패턴을 만드는 능력에서 갈린다.
    
    K리그 경기에서의 인사이트는 해당 시점의 선수 배치, 상대 압박, 공격의 전개 방향 등
    복합적인 맥락 속에서 ‘왜 그 공간으로 패스하는 것이 최적이었는가’를 이해하는 데서 나온다.

    특정 상황의 맥락을 AI가 학습하고, 이어지는 패스가 도달할 최적의 위치를 예측하고자 한다.

---

    본 프로젝트는 DACON과 함께합니다.

- [상세 설명](https://dacon.io/competitions/official/236647/overview/description)

## 2.1 도메인 지식



---

▸ 이벤트 좌표 사전 처리

    패스 방향은 항상 상대 공격 방향에 의해 정해지는데, 데이터는 이미 L->R 공격 방향으로 통일
    후반전에 공격 방향 반전되는 문제도 해결

▸ 풀백(포지션)

    축구 포지션 중에서 가장 패스 패턴이 뚜렷하게 나타나는 포지션
    측면(사이드 라인 근처)에 위치하고, 수비 + 공격 전개 둘 다 담당하는 포지션으로, 공간을 넓게 씀

▸ 패스의 4가지 의도

    1. 전진 패스(Progressive Pass): 상대 진영으로 전진해서 공격을 가속하기 위해
    2. 측면 전개(Switching / Wide Pass): 수비 압박을 벗어나기 위해
    3. 후방 안정화 패스(Safety / Reset): 볼 소유를 유지하고 다시 빌드업을 시작하기 위해
    4. 라인 침투 패스(Line Break Pass): 상대 수비 라인을 끊기 위해
    
    여기서 라인 침투 패스가 가장 가치가 높은 패스 !
    결국 패스를 예측하는 것은 위 4가지 중 어디에 속하는지를 모델이 판단하는 것과 같을 수 있음

▸ 패스 예측에서 중요한 특징들

| 유형    | 의미                          | 모델 영향         |
| ----- | --------------------------- | ------------- |
| 공간 정보 | start_x, start_y, 이전 이벤트 좌표 | 패스 방향성을 추론  |
| 압박 정보 | 주변에 상대 수비수가 있는가?            | 패스 거리/각도를 결정  |
| 경기 상황 | 점유 여부, 직전 이벤트 종류            | 패스의 의도를 추론      |

## 2.2 모델 선정

---

▸ Transformer Encoder

    위치 정보(sin positional encoding)가 좌표 시퀀스와 잘 맞고,
    long-range dependency가 LSTM보다 강함

    self-attention이 "어떤 이벤트가 패스 목적지 결정의 핵심인지" 자동으로 학습

    events → embedding → Transformer Encoder → MLP → (end_x, end_y)

▸ BiLSTM (입력 길이가 짧을 때 가장 안정적)

    학습이 Transformer보다 안정적이며 데이터 요구량이 낮음

▸ TCN (Temporal Convolutional Network)

    매우 빠르고, 좌표 이동 패턴을 convolution으로 잘 포착

# 3. 데이터 명세

## 3.1 DataFrame으로 명세 살펴보기

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

file_path = "Data/data_description.xlsx"

xls = pd.ExcelFile(file_path)
print("시트 목록:", xls.sheet_names)

sheet_names = xls.sheet_names

dfs = {}

for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    dfs[sheet] = df

### 3.1.1 match_info.csv 컬럼 정보

In [ ]:
dfs["컬럼 정의서 (Column Specs)"].iloc[:17]

### 3.1.2 train.csv, test.csv 컬럼 정보

In [ ]:
dfs["컬럼 정의서 (Column Specs)"].iloc[17:]

### 3.1.3 이벤트 타입 시트

In [ ]:
dfs["이벤트 타입 (Event Types)"]

## 3.2 match_info.csv



---

| column_name       | description           | data_type      | notes           |
| ----------------- | --------------------- | -------------- | --------------- |
| game_id           | 경기를 구분하는 고유 ID        | object         |                 |
| season_id         | 시즌 고유 ID              | object         | e.g., 2024시즌    |
| competition_id    | 대회 고유 ID              | object         | e.g., K리그1, FA컵 |
| game_day          | 리그 경기일 (e.g., 1R, 2R) | int64          | 'N라운드' 정보       |
| game_date         | 실제 경기 날짜              | datetime64[ns] |                 |
| home_team_id      | 홈 팀의 고유 ID            | object         |                 |
| away_team_id      | 어웨이 팀의 고유 ID          | object         |                 |
| home_score        | 홈 팀 득점                | int64          |                 |
| away_score        | 어웨이 팀 득점              | int64          |                 |
| venue             | 경기장 이름                | object         |                 |
| competition_name  | 대회 이름 (영문)            | object         |                 |
| country_name      | 리그 국가 이름              | object         |                 |
| season_name       | 시즌 이름                 | object         | e.g., '2024'    |
| home_team_name    | 홈 팀 이름 (영문)           | object         |                 |
| home_team_name_ko | 홈 팀 이름 (한글)           | object         |                 |
| away_team_name    | 어웨이 팀 이름 (영문)         | object         |                 |
| away_team_name_ko | 어웨이 팀 이름 (한글)         | object         |                 |

## 3.3 train / test.csv

---

| column_name  | description                       | data_type | notes                  |
| ------------ | --------------------------------- | --------- | ---------------------- |
| game_id      | 경기를 구분하는 고유 ID                    | int64     |                        |
| period_id    | 전/후반 구분 (1: 전반, 2: 후반…)           | int64     | 1부터 시작                 |
| episode_id   | 공이 라인 밖으로 나가기 전까지의 플레이 단위         | int64     | 1부터 시작                 |
| time_seconds | Period 시작 후 경과 시간(초)              | float64   | action_id와 순서 불일치 가능   |
| team_id      | 이벤트 수행 팀 ID                       | int64     |                        |
| player_id    | 이벤트 수행 선수 ID                      | float64   | 동시간대 이벤트 시 순서 뒤섞일 수 있음 |
| action_id    | 경기 내 이벤트 순서                       | int64     | 0부터 시작, 일부 이벤트 삭제됨     |
| type_name    | 이벤트 종류                            | object    | 이벤트 타입 표 참고            |
| result_name  | 이벤트 성공/실패 상태                      | object    | 이벤트 타입 표 참고            |
| start_x      | 이벤트 시작 X좌표                        | float64   | L→R 기준 좌표              |
| start_y      | 이벤트 시작 Y좌표                        | float64   | L→R 기준 좌표              |
| end_x        | 이벤트 종료 X좌표                        | float64   | L→R 기준 좌표              |
| end_y        | 이벤트 종료 Y좌표                        | float64   | L→R 기준 좌표              |
| is_home      | 홈 팀 여부                            | bool      |                        |
| game_episode | `{game_id}_{episode_id}` 고유 그룹 ID | object    |                        |

## 3.4 이벤트 타입(type_name)

---

| type_name        | result_name        | Description       | notes |
| ---------------- | ------------------ | ----------------- | ----- |
| Aerial Clearance | Successful         | 골키퍼 공중볼 처리 성공     |       |
| Aerial Clearance | Unsuccessful       | 공중볼 처리 실패         |       |
| Block            | NaN                | 상대 패스·크로스·슛 차단    |       |
| Carry            | NaN                | 공을 몰고 이동          |       |
| Catch            | NaN                | 골키퍼가 공을 캐치        |       |
| Clearance        | NaN                | 의도 없이 멀리 걷어낸 수비   |       |
| Cross            | Unsuccessful       | 크로스 실패            |       |
| Cross            | Successful         | 크로스 성공            |       |
| Deflection       | NaN                | 슛이 선수 몸에 맞아 방향 변함 |       |
| Duel             | Successful         | 경합 승리             |       |
| Duel             | Unsuccessful       | 경합 패배             |       |
| Error            | NaN                | 압박 없는 상황의 실수      |       |
| Foul             | NaN                | 파울(카드 없음)         |       |
| Foul             | Yellow_Card        | 옐로 카드 파울          |       |
| Foul             | Direct_Red_Card    | 다이렉트 레드           |       |
| Foul             | Second_Yellow_Card | 옐로 누적 레드          |       |
| Foul_Throw       | NaN                | 스로인 반칙            |       |
| Goal             | NaN                | 득점                |       |
| Goal Kick        | Successful         | 골킥 성공             |       |
| Goal Kick        | Unsuccessful       | 골킥 실패             |       |
| Handball_Foul    | NaN                | 핸드볼 파울            |       |
| Handball_Foul    | Yellow_Card        | 핸드볼 옐로카드          |       |
| Handball_Foul    | Second_Yellow_Card | 핸드볼 누적 레드         |       |
| Handball_Foul    | Direct_Red_Card    | 의도적 핸드볼 레드        |       |
| Hit              | NaN                | 의도치 않은 공 맞음       |       |
| Interception     | NaN                | 패스 경로 차단 후 소유권 획득 |       |
| Intervention     | NaN                | 소유권 방해(획득 X)      |       |
| Offside          | NaN                | 오프사이드             |       |
| Out              | NaN                | 공 아웃              |       |
| Own Goal         | NaN                | 자책골               |       |
| Parry            | NaN                | GK가 공을 쳐내기만 함     |       |
| Pass             | Successful         | 패스 성공             |       |
| Pass             | Unsuccessful       | 패스 실패             |       |
| Pass_Corner      | Unsuccessful       | 코너킥 패스 실패         |       |
| Pass_Corner      | Successful         | 코너킥 패스 성공         |       |
| Pass_Freekick    | Successful         | 프리킥 패스 성공         |       |
| Pass_Freekick    | Unsuccessful       | 프리킥 패스 실패         |       |
| Penalty Kick     | Goal               | PK 득점             |       |
| Penalty Kick     | On Target          | PK 유효슛, GK 방어     |       |
| Penalty Kick     | Off Target         | PK 빗나감            |       |
| Penalty Kick     | NaN                |                   |       |
| Recovery         | NaN                | 루즈볼 재획득           |       |
| Shot             | Off Target         | 골문 벗어난 슛          |       |
| Shot             | On Target          | 유효슈팅              |       |
| Shot             | Blocked            | 블록당함              |       |
| Shot             | Goal               | 슛 득점              |       |
| Shot             | Low Quality Shot   | 품질 낮은 슛           |       |
| Shot             | Keeper Rush-Out    | GK 돌출 상황 슛        |       |
| Shot             | NaN                |                   |       |
| Shot_Corner      | Goal               | 코너킥 직접 득점         |       |
| Shot_Freekick    | Off Target         | 프리킥 슛 빗나감         |       |
| Shot_Freekick    | Blocked            | 프리킥 블록            |       |
| Shot_Freekick    | On Target          | 프리킥 유효슛           |       |
| Shot_Freekick    | Goal               | 프리킥 득점            |       |
| Shot_Freekick    | Low Quality Shot   | 품질 낮은 프리킥 슛       |       |
| Tackle           | Unsuccessful       | 태클 실패             |       |
| Tackle           | Successful         | 태클 성공             |       |
| Take-On          | Unsuccessful       | 드리블 돌파 실패         |       |
| Take-On          | Successful         | 드리블 돌파 성공         |       |
| Throw-In         | Successful         | 스로인 성공            |       |
| Throw-In         | Unsuccessful       | 스로인 실패            |       |
| Throw-In         | NaN                |                   |       |

# 4. 데이터 살펴보기

## 4.1 match_info.csv

In [ ]:
file_path = 'Data/match_info.csv'

match_info = pd.read_csv(file_path)
match_info.head()

In [ ]:
match_info.info()



```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   game_id            228 non-null    int64
 1   season_id          228 non-null    int64
 2   competition_id     228 non-null    int64
 3   game_day           228 non-null    int64
 4   game_date          228 non-null    object
 5   home_team_id       228 non-null    int64
 6   away_team_id       228 non-null    int64
 7   home_score         228 non-null    int64
 8   away_score         228 non-null    int64
 9   venue              228 non-null    object
 10  competition_name   228 non-null    object
 11  country_name       228 non-null    object
 12  season_name        228 non-null    int64
 13  home_team_name     228 non-null    object
 14  home_team_name_ko  228 non-null    object
 15  away_team_name     228 non-null    object
 16  away_team_name_ko  228 non-null    object
dtypes: int64(9), object(8)
memory usage: 30.4+ KB
```



## 4.2 train.csv

In [ ]:
train_df = pd.read_csv('Data/train.csv')
train_df.info()



```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356721 entries, 0 to 356720
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   game_id       356721 non-null  int64  
 1   period_id     356721 non-null  int64  
 2   episode_id    356721 non-null  int64  
 3   time_seconds  356721 non-null  float64
 4   team_id       356721 non-null  int64  
 5   player_id     356721 non-null  int64  
 6   action_id     356721 non-null  int64  
 7   type_name     356721 non-null  object
 8   result_name   216467 non-null  object
 9   start_x       356721 non-null  float64
 10  start_y       356721 non-null  float64
 11  end_x         356721 non-null  float64
 12  end_y         356721 non-null  float64
 13  is_home       356721 non-null  bool   
 14  game_episode  356721 non-null  object
dtypes: bool(1), float64(5), int64(6), object(3)
memory usage: 38.4+ MB
```



In [ ]:
train_df.tail()

In [ ]:
last_events = (train_df.groupby('game_episode').tail(1))

not_pass_df = last_events[last_events['type_name'] != 'Pass']

print("총 에피소드 수 :", len(last_events))
print("마지막 이벤트가 Pass가 아닌 에피소드:", len(not_pass_df))
# display(not_pass_df.head())

    train도 검증
    
    총 에피소드 수 : 15435
    마지막 이벤트가 Pass가 아닌 에피소드: 0

In [ ]:
suc_pass_df = last_events[last_events['result_name'] == 'Successful']

print("패스에 성공한 에피소드 수 :", len(suc_pass_df))
print("패스에 실패한 에피소드 수 :", len(last_events) - len(suc_pass_df))

    패스에 성공한 에피소드 수 : 8634
    패스에 실패한 에피소드 수 : 6801

## 4.3 test.csv

In [ ]:
test_df = pd.read_csv('Data/test.csv')
test_df.info()



```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2414 entries, 0 to 2413
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   game_id       2414 non-null   int64
 1   game_episode  2414 non-null   object
 2   path          2414 non-null   object
dtypes: int64(1), object(2)
memory usage: 56.7+ KB
```



In [ ]:
test_df.head()

In [ ]:
# 내부 파일(path)를 따라 하나만 출력해보자
path = 'Data' + test_df.iloc[0]['path'][1:]

epi1_test_df = pd.read_csv(path)

# 예측해야할 것 = 최종 좌표
print(epi1_test_df.iloc[-1]['end_x'], epi1_test_df.iloc[-1]['end_y'])

In [ ]:
epi1_test_df.tail(1)

    그렇다면, 최종 좌표들이 모두 '패스' 좌표인가 ??! 검증해보자.

In [ ]:
from tqdm import tqdm

not_pass_list = []

for path in tqdm(test_df['path'], desc="Checking episodes"):
    full_path = 'Data' + path[1:]
    epi_df = pd.read_csv(full_path)

    if epi_df.iloc[-1]['type_name'] != 'Pass':
        not_pass_list.append(full_path)

not_pass_list

    검증 결과,

    Checking episodes: 100%|██████████| 2414/2414 [05:31<00:00,  7.29it/s]
    [] -> 빈 리스트라면, 모두 Pass라는 뜻

    최종 이벤트는 모두 패스인 것을 확인


In [ ]:
suc_pass_list = []

for path in tqdm(test_df['path'], desc="Checking episodes"):
    full_path = 'Data' + path[1:]
    epi_df = pd.read_csv(full_path)

    if epi_df.iloc[-1]['result_name'] == 'Successful':
        suc_pass_list.append(full_path)

In [ ]:
print(f'성공한 최종 패스: {len(suc_pass_list)}개, 실패한 최종 패스: {len(test_df)-len(suc_pass_list)}개')

    성공한 최종 패스: 1317개, 실패한 최종 패스: 1097개

    결국 예측해야할 것은 에피소드별 최종 'Pass' 좌표 !
    다른 이벤트들의 좌표들은 주어져있기 때문에,
    최종 좌표, 그 이전의 어떤 좌표와 맥락을 띠고 있었는지를 함께 담아주는 seq-to-regression 모델을 선정해야할 것 같음

    예를 들어 간단하게 BiLSTM, Transformer 구조로 baseline을 잡아야 할 듯 !

# 5. 문제 해결 프로세스 정의

---

▸ 문제
  
    1. As-Is
    기존 분석은 단편적인 이벤트 기록에 그쳐 전술적 의도 해석이 어렵다.

    2. To-Be
    Episode를 기반으로 AI가 플레이 맥락을 학습하는 구조를 구축한다.

    3. Goal
    주어진 시퀀스(episode)의 마지막 패스 좌표(end_x, end_y)를 예측

▸ 기대 효과

    전술 분석 자동화 및 선수/팀 평가 고도화
    데이터 기반 경기 해석 능력 향상

▸ 해결 방안

    시퀀스 중심의 Transformer/LSTM 기반 모델 적용
    Δx/Δy, 패스 각도, zone 등 도메인 기반 feature engineering
    train/test episode reconstruction 및 통합 전처리 파이프라인 만들기

▸ 성과 측정

    MAE/RMSE 기반 좌표 예측 정확도 평가
    zone 단위 tactical accuracy 분석
    episode length별 성능 비교

    결과적으로 유클리드 거리(Euclidean Distance)를 이용한 평가 진행


# 6. Feature Engineering

---

▸ 패스 각도(angle)

    angle = arctan((end_y-start_y) / (end_x-start_x))

    풀백은 측면으로 많이 주고, 중앙 미드필더는 전진 패스의 비율 높음
    수비수는 옆으로 주는 패스나 후방 패스의 비중 높음

▸ 패스 진행 거리

    더 먼 패스일수록 progressive chance가 높고, end_x가 강하게 증가하는 패턴을 가짐

▸ event_type 임베딩

    type_name → embedding vector
    result_name → embedding vector

    sequence embedding에 필수적으로 진행해야하는 것

▸ 에피소드에서의 속도(Δx, Δy)

    dx_t = x_t - x_(t-1)
    dy_t = y_t - y_(t-1)

    엔드 투 엔드 모델보다 훨씬 패턴 학습이 잘 됨

    dx > 0 → 오른쪽으로 전진 중
    dy > 0 → 위쪽으로 이동 중
    dy < 0 → 아래쪽으로 이동 중
    dx ≈ 0 → 횡패스 빈도 높음
    dx < 0 → 후방 패스 비율 증가 (안정화)

    1. 한 에피소드에서 dx가 계속 증가한다 ➜ 공격 전개 중 (전진 패스 가능성이 높음)
    2. dy가 크게 증가했다➜ 측면 전개 중 (사이드로 패스가 날아갈 가능성)
    3. dx가 음수로 전환되었다 ➜ 후방 안정화 패스 패턴
    4. dx, dy가 급격히 바뀐다 ➜ 압박을 벗어나기 위한 빠른 전개